** For google colab**



# Finding deltas for v4

In [1]:
def deltas_in_search(file_name):
    file = open(file_name, 'r')

    count = 0
    for line in file:
        count +=1
        if count == 5:
            break
        continue

    deltas = {}
    upper = {}
    
    for line in file:
        if line.find('Searching') != -1:
            J0 = line.split()[1]
            deltas[J0] = []
            E0 = float(line.split()[-1])
            continue

        if line.find('No') != -1 or \
           len(line.split()) == 0 or \
           int(line.split()[0]) != int(J0) - 1:
            continue
        
        try:
            E = float(line.split()[-1])
            delta = round(E - E0, 3)
            deltas[J0].append(delta)
        except ValueError:
            break
    return deltas, indexes


In [6]:
from google.colab import files

uploaded = files.upload()
deltas, indexes = deltas_in_search('search_J1J-1')
print(deltas)

print(len(deltas))
counter = 1
for k,v in deltas.items():
  counter *= len(v)
  
print(counter)

Saving search_J1J-1 to search_J1J-1
{'1': [-0.493, -0.377, -0.302, -0.232, -0.208, -0.063, 0.101, 0.183, 0.188, 0.292, 0.313, 0.318, 0.451], '2': [-0.493, -0.44, -0.435, -0.4, -0.383, -0.38, -0.351, -0.293, -0.266, -0.231, -0.225, -0.197, -0.176, -0.158, -0.123, -0.109, -0.06, -0.05, -0.043, -0.026, 0.001, 0.026, 0.033, 0.046, 0.051, 0.056, 0.09, 0.099, 0.146, 0.219, 0.224, 0.284, 0.413, 0.486, 0.495], '3': [-0.499, -0.494, -0.476, -0.462, -0.406, -0.387, -0.375, -0.345, -0.334, -0.326, -0.32, -0.309, -0.299, -0.272, -0.26, -0.2, -0.17, -0.136, -0.131, -0.125, -0.12, -0.095, -0.074, -0.041, -0.035, -0.008, 0.035, 0.041, 0.046, 0.066, 0.071, 0.09, 0.143, 0.156, 0.166, 0.199, 0.251, 0.257, 0.295, 0.301, 0.306, 0.316, 0.367, 0.434, 0.472, 0.479], '4': [-0.46, -0.427, -0.382, -0.372, -0.352, -0.335, -0.245, -0.226, -0.155, -0.138, -0.096, -0.034, 0.06, 0.086, 0.113, 0.153, 0.157, 0.172, 0.178, 0.226, 0.287, 0.33, 0.344, 0.35, 0.36, 0.483, 0.494], '5': [-0.508, -0.499, -0.425, -0.384, -0.35

In [0]:
keys = list(deltas.keys())
ini_key = keys[0]
seq_ini = []
sequence = []
data = []

def tree_test(k, seq_ini):
    for v in deltas[k]:
        sequence = seq_ini.copy()
        
        if len(seq_ini) != 0:
            if sequence[-1] > v:
                continue
            else:
                sequence.append(v)
        else:
            sequence.append(v)
        
        if len(sequence) == len(deltas.items()):
                data.append(sequence)
                #print(sequence)
        else:
                tree_test(str(int(k)+1), sequence)

        

tree_test(ini_key,sequence)
  

In [12]:
print(len(data))
print(data[-10:])

1656681
[[0.188, 0.284, 0.316, 0.36, 0.411, 0.436], [0.188, 0.284, 0.316, 0.36, 0.411, 0.464], [0.188, 0.284, 0.316, 0.36, 0.411, 0.479], [0.188, 0.284, 0.316, 0.36, 0.411, 0.483], [0.188, 0.284, 0.316, 0.36, 0.429, 0.436], [0.188, 0.284, 0.316, 0.36, 0.429, 0.464], [0.188, 0.284, 0.316, 0.36, 0.429, 0.479], [0.188, 0.284, 0.316, 0.36, 0.429, 0.483], [0.188, 0.284, 0.316, 0.36, 0.467, 0.479], [0.188, 0.284, 0.316, 0.36, 0.467, 0.483]]


# Y = 1 Labels from v7

In [0]:
import numpy as np

# Write output csv file

In [9]:
from google.colab import files

#import file csv
uploaded = files.upload()


Saving test.csv to test.csv


In [0]:
import csv
import random
random.seed(227)

file_csv = open('test.csv', 'w', newline='')

filewriter = csv.writer(file_csv, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
filewriter.writerow([1,2,3,4,5,6])
for i in range(int(len(data)/2), len(data)):
    filewriter.writerow(data[i])

#files.download('train.csv')

In [0]:
file_csv.close()
files.download('test.csv')


In [5]:
from keras.models import Model
from keras.layers import Dense, Input, LSTM

def model(input_shape, Ty):
    i = Input(shape=input_shape, dtype='float32')
    X = LSTM(32, return_sequences=False)(i)
    X = Dense(Ty, activation='sigmoid')(X)
    #X = Activation('softmax')(X)
    model = Model(inputs=[i], outputs=X)
    
    return model

Tx, Ty =  6, 1

LSTM_model = model((Tx,1),Ty)
LSTM_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

LSTM_model.load_weights('sources/LSTM_sequence_weights.h5')